# 09 Merge Data with Calendar (Dates) and Transfer

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sklearn
from datetime import datetime
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import shapely
import hashlib

In [2]:
# Set month
data_month_single = '7'
data_month_double = '07'
data_month_int = 7

## Load so far merged data

In [3]:
# load so far merged data
#realtime = pd.read_csv('../Data/09_first_merge_1_NEU_NEU.csv')
realtime = pd.read_csv('../Data/08_realtime_buffer_delay)

/tmp/ipykernel_2687996/273967643.py:2: DtypeWarning: Columns (3,17) have mixed types. Specify dtype option on import or set low_memory=False.
  realtime = pd.read_csv('../Data/09_first_merge_1_NEU_NEU.csv')


In [4]:
# transform geometry
#point = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(realtime["station_point"], crs=4326))
#buffer = gpd.GeoDataFrame(geometry=gpd.GeoSeries.from_wkb(realtime["buffer_zone"], crs=4326))

In [5]:
# drop geometry columns in wrong format
#realtime = realtime.drop(columns=["station_point", "buffer_zone"])

In [6]:
# replace geometry columns with correct format
#realtime['station_point'] = point
#realtime['buffer_zone'] = buffer

In [7]:
#realtime.info()

In [8]:
#realtime

In [9]:
unique_service_ids = realtime['service_id'].unique().tolist()

In [10]:
## Calendar

In [11]:
# load calendar data
calendar = pd.DataFrame(pd.read_csv("../Data/Raw/20230701-000000-UTC_gtfs-scraper_calendar-vrs.csv"))
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,3,1,1,1,1,1,1,1,20221211,20231209
1,5,1,1,1,1,1,1,1,20221211,20231209
2,11,1,1,1,1,1,1,1,20221211,20231209
3,22,1,1,1,1,1,0,1,20221211,20231209
4,42,1,1,1,1,1,1,0,20221211,20231209
...,...,...,...,...,...,...,...,...,...,...
4404,29242,1,1,1,1,1,1,1,20221211,20231209
4405,29243,0,0,0,0,0,1,1,20221211,20231209
4406,29245,0,0,0,0,1,0,0,20221211,20231209
4407,29246,0,0,0,0,1,0,0,20221211,20231209


In [12]:
# reduce to data within already merged data
filtered_calendar = calendar[calendar['service_id'].isin(unique_service_ids)]

In [13]:
filtered_calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,3,1,1,1,1,1,1,1,20221211,20231209
3,22,1,1,1,1,1,0,1,20221211,20231209
5,43,1,1,1,1,1,1,0,20221211,20231209
7,61,1,1,1,1,1,0,0,20221211,20231209
9,64,1,1,1,1,1,0,0,20221211,20231209
...,...,...,...,...,...,...,...,...,...,...
4342,29130,1,1,1,1,1,0,1,20221211,20231209
4343,29131,1,1,1,1,0,1,1,20221211,20231209
4344,29132,1,1,1,1,1,0,0,20221211,20231209
4354,29144,1,1,1,1,1,0,0,20221211,20231209


In [14]:
# create new column in datetime format - important for later operation 
filtered_calendar['start'] = pd.to_datetime(filtered_calendar['start_date'], format='%Y%m%d')
filtered_calendar['end'] = pd.to_datetime(filtered_calendar['end_date'], format='%Y%m%d')

/tmp/ipykernel_2687996/2526336632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_calendar['start'] = pd.to_datetime(filtered_calendar['start_date'], format='%Y%m%d')
/tmp/ipykernel_2687996/2526336632.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_calendar['end'] = pd.to_datetime(filtered_calendar['end_date'], format='%Y%m%d')


In [15]:
filtered_calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start,end
0,3,1,1,1,1,1,1,1,20221211,20231209,2022-12-11,2023-12-09
3,22,1,1,1,1,1,0,1,20221211,20231209,2022-12-11,2023-12-09
5,43,1,1,1,1,1,1,0,20221211,20231209,2022-12-11,2023-12-09
7,61,1,1,1,1,1,0,0,20221211,20231209,2022-12-11,2023-12-09
9,64,1,1,1,1,1,0,0,20221211,20231209,2022-12-11,2023-12-09
...,...,...,...,...,...,...,...,...,...,...,...,...
4342,29130,1,1,1,1,1,0,1,20221211,20231209,2022-12-11,2023-12-09
4343,29131,1,1,1,1,0,1,1,20221211,20231209,2022-12-11,2023-12-09
4344,29132,1,1,1,1,1,0,0,20221211,20231209,2022-12-11,2023-12-09
4354,29144,1,1,1,1,1,0,0,20221211,20231209,2022-12-11,2023-12-09


In [16]:
# create a copy of realtime to not mess up the original data
merged_df = realtime
merged_df['service'] = 0 # create new column for service


In [17]:
# create columnd for date and weekday
merged_df['date'] = pd.to_datetime(merged_df['scheduled_arrival_time']).dt.date
merged_df['weekday'] = pd.to_datetime(merged_df['scheduled_arrival_time']).dt.day_name()

In [18]:
merged_df

,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,...,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday
0,0,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Bad Godesberg Stadthalle,POINT (7.158347000000002 50.680336000000004),"POLYGON ((7.161095563015469 50.68066886560957,...",2023-07-01 05:17:00,2023-07-01 05:17:00,NaN,0 days,0,2023-07-01,Saturday
1,1,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Bad Godesberg Bf,POINT (7.159553000000001 50.68392),"POLYGON ((7.162301786678823 50.68425285400936,...",2023-07-01 05:18:00,2023-07-01 05:18:00,0 days,0 days,0,2023-07-01,Saturday
2,2,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Plittersdorfer Str.,POINT (7.153876000000002 50.687850999999995),POLYGON ((7.1566249492160345 50.68818400637061...,2023-07-01 05:19:00,2023-07-01 05:19:00,0 days,0 days,0,2023-07-01,Saturday
3,3,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Wurzerstr.,POINT (7.1482320000000055 50.691395),"POLYGON ((7.150981089535626 50.69172815614405,...",2023-07-01 05:20:00,2023-07-01 05:20:00,0 days,0 days,0,2023-07-01,Saturday
4,4,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Hochkreuz/Deutsches Museum Bonn,POINT (7.141498000000006 50.69677399999999),POLYGON ((7.1442473242318325 50.69710734026554...,2023-07-01 05:21:00,2023-07-01 05:21:00,0 days,0 days,0,2023-07-01,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71174467,71174467,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,Bonn Am Bruch,POINT (7.048342000000008 50.718540999999995),"POLYGON ((7.051091489774908 50.71887664006563,...",2023-07-31 20:36:00,2023-07-31 20:36:00,NaN,NaN,0,2023-07-31,Monday
71174468,71174468,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,Bonn Roncallistr.,POINT (7.044228999999996 50.722075000000004),POLYGON ((7.0469786475110325 50.72241075379365...,2023-07-31 20:37:00,2023-07-31 20:37:00,NaN,NaN,0,2023-07-31,Monday
71174469,71174469,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,Bonn Bonner Logsweg,POINT (7.045797000000002 50.723193999999985),"POLYGON ((7.048546731723949 50.72352972210798,...",2023-07-31 20:38:00,2023-07-31 20:38:00,NaN,NaN,0,2023-07-31,Monday
71174470,71174470,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,Bonn Malgisostr.,POINT (7.047697000000004 50.722637),POLYGON ((7.050446721797707 50.722972674659836...,2023-07-31 20:38:00,2023-07-31 20:38:00,NaN,NaN,0,2023-07-31,Monday


In [19]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71174472 entries, 0 to 71174471
Data columns (total 28 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   Unnamed: 0                int64 
 1   route_id                  int64 
 2   agency_id                 int64 
 3   route_short_name          object
 4   route_type                int64 
 5   route_type_name           object
 6   agency_name               object
 7   service_id                int64 
 8   trip_id                   object
 9   trip_headsign             object
 10  direction_id              int64 
 11  shape_id                  int64 
 12  stop_id                   int64 
 13  actual_arrival_time       object
 14  actual_departure_time     object
 15  vrs_timestamp             int64 
 16  stop_sequence             int64 
 17  stop_headsign             object
 18  stop_name                 object
 19  station_point             object
 20  buffer_zone               object
 21  schedu

In [20]:
merged_df['scheduled_arrival_time'] = pd.to_datetime(merged_df['scheduled_arrival_time'])
merged_df['scheduled_departure_time'] = pd.to_datetime(merged_df['scheduled_departure_time'])
merged_df['actual_departure_time'] = pd.to_datetime(merged_df['actual_departure_time'])
merged_df['actual_arrival_time'] = pd.to_datetime(merged_df['actual_arrival_time'])

In [21]:
merged_df

,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,...,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday
0,0,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Bad Godesberg Stadthalle,POINT (7.158347000000002 50.680336000000004),"POLYGON ((7.161095563015469 50.68066886560957,...",2023-07-01 05:17:00,2023-07-01 05:17:00,NaN,0 days,0,2023-07-01,Saturday
1,1,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Bad Godesberg Bf,POINT (7.159553000000001 50.68392),"POLYGON ((7.162301786678823 50.68425285400936,...",2023-07-01 05:18:00,2023-07-01 05:18:00,0 days,0 days,0,2023-07-01,Saturday
2,2,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Plittersdorfer Str.,POINT (7.153876000000002 50.687850999999995),POLYGON ((7.1566249492160345 50.68818400637061...,2023-07-01 05:19:00,2023-07-01 05:19:00,0 days,0 days,0,2023-07-01,Saturday
3,3,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Wurzerstr.,POINT (7.1482320000000055 50.691395),"POLYGON ((7.150981089535626 50.69172815614405,...",2023-07-01 05:20:00,2023-07-01 05:20:00,0 days,0 days,0,2023-07-01,Saturday
4,4,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,Bonn Hochkreuz/Deutsches Museum Bonn,POINT (7.141498000000006 50.69677399999999),POLYGON ((7.1442473242318325 50.69710734026554...,2023-07-01 05:21:00,2023-07-01 05:21:00,0 days,0 days,0,2023-07-01,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71174467,71174467,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,Bonn Am Bruch,POINT (7.048342000000008 50.718540999999995),"POLYGON ((7.051091489774908 50.71887664006563,...",2023-07-31 20:36:00,2023-07-31 20:36:00,NaN,NaN,0,2023-07-31,Monday
71174468,71174468,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,Bonn Roncallistr.,POINT (7.044228999999996 50.722075000000004),POLYGON ((7.0469786475110325 50.72241075379365...,2023-07-31 20:37:00,2023-07-31 20:37:00,NaN,NaN,0,2023-07-31,Monday
71174469,71174469,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,Bonn Bonner Logsweg,POINT (7.045797000000002 50.723193999999985),"POLYGON ((7.048546731723949 50.72352972210798,...",2023-07-31 20:38:00,2023-07-31 20:38:00,NaN,NaN,0,2023-07-31,Monday
71174470,71174470,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,Bonn Malgisostr.,POINT (7.047697000000004 50.722637),POLYGON ((7.050446721797707 50.722972674659836...,2023-07-31 20:38:00,2023-07-31 20:38:00,NaN,NaN,0,2023-07-31,Monday


In [22]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71174472 entries, 0 to 71174471
Data columns (total 28 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Unnamed: 0                int64         
 1   route_id                  int64         
 2   agency_id                 int64         
 3   route_short_name          object        
 4   route_type                int64         
 5   route_type_name           object        
 6   agency_name               object        
 7   service_id                int64         
 8   trip_id                   object        
 9   trip_headsign             object        
 10  direction_id              int64         
 11  shape_id                  int64         
 12  stop_id                   int64         
 13  actual_arrival_time       datetime64[ns]
 14  actual_departure_time     datetime64[ns]
 15  vrs_timestamp             int64         
 16  stop_sequence             int64         
 17  stop_h

In [23]:
# rename columns for later operation
filtered_calendar.rename(columns=lambda x: x.capitalize(), inplace=True)
filtered_calendar.rename(columns={'Service_id': 'service_id'}, inplace=True)

/tmp/ipykernel_2687996/3862232944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_calendar.rename(columns=lambda x: x.capitalize(), inplace=True)
/tmp/ipykernel_2687996/3862232944.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_calendar.rename(columns={'Service_id': 'service_id'}, inplace=True)


In [24]:
filtered_calendar

,service_id,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Start_date,End_date,Start,End
0,3,1,1,1,1,1,1,1,20221211,20231209,2022-12-11,2023-12-09
3,22,1,1,1,1,1,0,1,20221211,20231209,2022-12-11,2023-12-09
5,43,1,1,1,1,1,1,0,20221211,20231209,2022-12-11,2023-12-09
7,61,1,1,1,1,1,0,0,20221211,20231209,2022-12-11,2023-12-09
9,64,1,1,1,1,1,0,0,20221211,20231209,2022-12-11,2023-12-09
...,...,...,...,...,...,...,...,...,...,...,...,...
4342,29130,1,1,1,1,1,0,1,20221211,20231209,2022-12-11,2023-12-09
4343,29131,1,1,1,1,0,1,1,20221211,20231209,2022-12-11,2023-12-09
4344,29132,1,1,1,1,1,0,0,20221211,20231209,2022-12-11,2023-12-09
4354,29144,1,1,1,1,1,0,0,20221211,20231209,2022-12-11,2023-12-09


In [25]:
# check, if there are different start and end times in calendar data
check_date = filtered_calendar[filtered_calendar['End'] != '2023-12-09']
check_date

,service_id,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Start_date,End_date,Start,End


In [26]:
n = len(merged_df) // 3

In [27]:
merged1 = merged_df.iloc[:n]          # First n rows
merged2 = merged_df.iloc[n:2*n]       # Next n rows
merged3 = merged_df.iloc[2*n:]   

All service information lies in the timeframe 2022-12-11 and 2023-12-09

In [28]:
# check for every entry in merged_df the corresponding entry in filtered data
count = 0

print('Init')

for index, row in merged1.iterrows():
    
    count = count+1
    
    service_id = row['service_id']
    scheduled_arrival_time = row['scheduled_arrival_time']
    scheduled_departure_time = row['scheduled_departure_time']
    weekday = row['weekday']
    
    if count % 100_000 == 0:
        print(str(count) + ' from ' + str(len(merged1)) + ' rows' )
    if count == len(merged1):
        print('End')

    
    
    # Check if service_id is in filtered_calendar
    if service_id in filtered_calendar['service_id'].values:
        # Find the corresponding row in filtered_calendar
        calendar_row = filtered_calendar[filtered_calendar['service_id'] == service_id]
        
        # Check if scheduled_arrival_time and scheduled_departure_time lie within start and end dates
        if (scheduled_arrival_time >= calendar_row['Start'].values[0]) and (scheduled_departure_time <= calendar_row['End'].values[0]):
            # Check if weekday has an entry in filtered_calendar
            if calendar_row[weekday].values[0] == 1:
                merged1.at[index, 'service'] = 1

Init
100000 from 23724824 rows
200000 from 23724824 rows
300000 from 23724824 rows
400000 from 23724824 rows
500000 from 23724824 rows
600000 from 23724824 rows
700000 from 23724824 rows
800000 from 23724824 rows
900000 from 23724824 rows
1000000 from 23724824 rows
1100000 from 23724824 rows
1200000 from 23724824 rows
1300000 from 23724824 rows
1400000 from 23724824 rows
1500000 from 23724824 rows
1600000 from 23724824 rows
1700000 from 23724824 rows
1800000 from 23724824 rows
1900000 from 23724824 rows
2000000 from 23724824 rows
2100000 from 23724824 rows
2200000 from 23724824 rows
2300000 from 23724824 rows
2400000 from 23724824 rows
2500000 from 23724824 rows
2600000 from 23724824 rows
2700000 from 23724824 rows
2800000 from 23724824 rows
2900000 from 23724824 rows
3000000 from 23724824 rows
3100000 from 23724824 rows
3200000 from 23724824 rows
3300000 from 23724824 rows
3400000 from 23724824 rows
3500000 from 23724824 rows
3600000 from 23724824 rows
3700000 from 23724824 rows
38000

In [30]:
#merged1.to_csv('../Data/10_merged1.csv')
merged1.to_csv('../Data/merged1.csv)

In [36]:
# check for every entry in merged_df the corresponding entry in filtered data
count = 0

for index, row in merged2.iterrows():
    
    count = count+1
    
    service_id = row['service_id']
    scheduled_arrival_time = row['scheduled_arrival_time']
    scheduled_departure_time = row['scheduled_departure_time']
    weekday = row['weekday']
    
    if count % 100_000 == 0:
        print(str(count) + ' from ' + str(len(merged1)) + ' rows' )
    if count == len(merged1):
        print('End')
    
    # Check if service_id is in filtered_calendar
    if service_id in filtered_calendar['service_id'].values:
        # Find the corresponding row in filtered_calendar
        calendar_row = filtered_calendar[filtered_calendar['service_id'] == service_id]
        
        # Check if scheduled_arrival_time and scheduled_departure_time lie within start and end dates
        if (scheduled_arrival_time >= calendar_row['Start'].values[0]) and (scheduled_departure_time <= calendar_row['End'].values[0]):
            # Check if weekday has an entry in filtered_calendar
            if calendar_row[weekday].values[0] == 1:
                merged2.at[index, 'service'] = 1
                

100000 from 23724824 rows
200000 from 23724824 rows
300000 from 23724824 rows
400000 from 23724824 rows
500000 from 23724824 rows
600000 from 23724824 rows
700000 from 23724824 rows
800000 from 23724824 rows
900000 from 23724824 rows
1000000 from 23724824 rows
1100000 from 23724824 rows
1200000 from 23724824 rows
1300000 from 23724824 rows
1400000 from 23724824 rows
1500000 from 23724824 rows
1600000 from 23724824 rows
1700000 from 23724824 rows
1800000 from 23724824 rows
1900000 from 23724824 rows
2000000 from 23724824 rows
2100000 from 23724824 rows
2200000 from 23724824 rows
2300000 from 23724824 rows
2400000 from 23724824 rows
2500000 from 23724824 rows
2600000 from 23724824 rows
2700000 from 23724824 rows
2800000 from 23724824 rows
2900000 from 23724824 rows
3000000 from 23724824 rows
3100000 from 23724824 rows
3200000 from 23724824 rows
3300000 from 23724824 rows
3400000 from 23724824 rows
3500000 from 23724824 rows
3600000 from 23724824 rows
3700000 from 23724824 rows
3800000 fr

In [38]:
#merged2.to_csv('../Data/10_merged2.csv')
merged2.to_csv('../Data/merged2.csv')

In [39]:
# check for every entry in merged_df the corresponding entry in filtered data
count = 0

for index, row in merged3.iterrows():
    
    count = count+1
    
    service_id = row['service_id']
    scheduled_arrival_time = row['scheduled_arrival_time']
    scheduled_departure_time = row['scheduled_departure_time']
    weekday = row['weekday']
    
    if count % 100_000 == 0:
        print(str(count) + ' from ' + str(len(merged1)) + ' rows' )
    if count == len(merged1):
        print('End')
    
    # Check if service_id is in filtered_calendar
    if service_id in filtered_calendar['service_id'].values:
        # Find the corresponding row in filtered_calendar
        calendar_row = filtered_calendar[filtered_calendar['service_id'] == service_id]
        
        # Check if scheduled_arrival_time and scheduled_departure_time lie within start and end dates
        if (scheduled_arrival_time >= calendar_row['Start'].values[0]) and (scheduled_departure_time <= calendar_row['End'].values[0]):
            # Check if weekday has an entry in filtered_calendar
            if calendar_row[weekday].values[0] == 1:
                merged3.at[index, 'service'] = 1

100000 from 23724824 rows
200000 from 23724824 rows
300000 from 23724824 rows
400000 from 23724824 rows
500000 from 23724824 rows
600000 from 23724824 rows
700000 from 23724824 rows
800000 from 23724824 rows
900000 from 23724824 rows
1000000 from 23724824 rows
1100000 from 23724824 rows
1200000 from 23724824 rows
1300000 from 23724824 rows
1400000 from 23724824 rows
1500000 from 23724824 rows
1600000 from 23724824 rows
1700000 from 23724824 rows
1800000 from 23724824 rows
1900000 from 23724824 rows
2000000 from 23724824 rows
2100000 from 23724824 rows
2200000 from 23724824 rows
2300000 from 23724824 rows
2400000 from 23724824 rows
2500000 from 23724824 rows
2600000 from 23724824 rows
2700000 from 23724824 rows
2800000 from 23724824 rows
2900000 from 23724824 rows
3000000 from 23724824 rows
3100000 from 23724824 rows
3200000 from 23724824 rows
3300000 from 23724824 rows
3400000 from 23724824 rows
3500000 from 23724824 rows
3600000 from 23724824 rows
3700000 from 23724824 rows
3800000 fr

In [41]:
#merged3.to_csv('../Data/10_merged3.csv')
merged3.to_csv('../Data/merged3.csv')

In [42]:
merge_1 = pd.concat([merged1, merged2])

In [43]:
merge_2 = pd.concat([merge_1, merged3])

In [44]:
merged_df = merge_2

In [45]:
merged_df[merged_df['service'] == 0]

,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,...,stop_name,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday
44757,44757,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11467,231-16-001-687.4.13:283300-12-257_62FF815A-C96...,Wesseling,...,Bonn Hbf,POINT (7.097127999999999 50.732008),POLYGON ((7.0998788594577125 50.73234255286602...,2023-07-01 04:33:00,2023-07-01 04:33:00,NaN,0 days,0,2023-07-01,Saturday
44758,44758,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11467,231-16-001-687.4.13:283300-12-257_62FF815A-C96...,Wesseling,...,Bonn West,POINT (7.081126999999997 50.736562000000006),"POLYGON ((7.083877935788258 50.73689695186694,...",2023-07-01 04:35:00,2023-07-01 04:35:00,0 days,0 days,0,2023-07-01,Saturday
44759,44759,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11467,231-16-001-687.4.13:283300-12-257_62FF815A-C96...,Wesseling,...,Bonn Propsthof Nord,POINT (7.073506999999998 50.739096),"POLYGON ((7.076257993477945 50.73943114361114,...",2023-07-01 04:36:00,2023-07-01 04:36:00,0 days,0 days,0,2023-07-01,Saturday
44760,44760,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11467,231-16-001-687.4.13:283300-12-257_62FF815A-C96...,Wesseling,...,Bonn Tannenbusch Süd,POINT (7.055560999999996 50.742369999999994),"POLYGON ((7.058311971324935 50.74270558248207,...",2023-07-01 04:38:00,2023-07-01 04:38:00,0 days,0 days,0,2023-07-01,Saturday
44761,44761,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,11467,231-16-001-687.4.13:283300-12-257_62FF815A-C96...,Wesseling,...,Bonn Tannenbusch Mitte,POINT (7.0465420000000005 50.750893000000005),"POLYGON ((7.049293363220795 50.75122883557303,...",2023-07-01 04:40:00,2023-07-01 04:40:00,0 days,0 days,0,2023-07-01,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70988257,70988257,857,12,857,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,28775,1661-857-012-1489.2.21:244300-38-1_748DB621-A7...,Bad Godesberg Bf / Löbestr.,...,Bonn Theodor-Heuss-Str.,POINT (7.160062000000005 50.679294),"POLYGON ((7.162810522432749 50.67962682026901,...",2023-07-30 01:40:00,2023-07-30 01:40:00,NaN,0 days,0,2023-07-30,Sunday
70988258,70988258,857,12,857,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,28775,1661-857-012-1489.2.21:244300-38-1_748DB621-A7...,Bad Godesberg Bf / Löbestr.,...,Bonn Bad Godesberg Stadthalle,POINT (7.158347000000002 50.680336000000004),"POLYGON ((7.161095563015469 50.68066886560957,...",2023-07-30 01:42:00,2023-07-30 01:42:00,NaN,0 days,0,2023-07-30,Sunday
70988259,70988259,857,12,857,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,28775,1661-857-012-1489.2.21:244300-38-1_748DB621-A7...,Bad Godesberg Bf / Löbestr.,...,Bonn Rigalsche Wiese,POINT (7.156307 50.67907100000001),"POLYGON ((7.159055464982307 50.67940390777223,...",2023-07-30 01:43:00,2023-07-30 01:43:00,NaN,0 days,0,2023-07-30,Sunday
70988260,70988260,857,12,857,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,28775,1661-857-012-1489.2.21:244300-38-1_748DB621-A7...,Bad Godesberg Bf / Löbestr.,...,Bonn Am Kurpark,POINT (7.1555750000000025 50.68328499999999),POLYGON ((7.158323702507419 50.683617944840954...,2023-07-30 01:44:00,2023-07-30 01:44:00,NaN,0 days,0,2023-07-30,Sunday


## Transfer

In [46]:
#transfers = pd.DataFrame(pd.read_csv("../Data/Raw/20230701-000000-UTC_gtfs-scraper_transfers-vrs.csv"))
transfers = pd.DataFrame(pd.read_csv('../Data/gtfs-scraper/20230701-000000-UTC_gtfs-scraper_transfers-vrs.csv'))
transfers


,from_stop_id,to_stop_id,transfer_type,min_transfer_time
0,1,1,2,300.0
1,2,2,2,300.0
2,3,3,2,300.0
3,3,6812,2,240.0
4,4,4,2,300.0
...,...,...,...,...
1519,11058,11058,2,240.0
1520,11108,11108,2,300.0
1521,11158,11158,2,300.0
1522,17795,9618,2,480.0


In [47]:
filtered_transfers = transfers[transfers['from_stop_id'] != transfers['to_stop_id']]

In [48]:
filtered_transfers

,from_stop_id,to_stop_id,transfer_type,min_transfer_time
3,3,6812,2,240.0
8,8,9,2,420.0
9,8,752,2,420.0
10,9,8,2,420.0
26,25,26,2,600.0
...,...,...,...,...
1424,9712,692,2,360.0
1425,9758,9601,2,300.0
1426,9861,6625,2,420.0
1522,17795,9618,2,480.0


In [49]:
merged_df['transfer_stop'] = merged_df['stop_id'].isin(filtered_transfers['from_stop_id'])
merged_df['transfer_stop'] = merged_df['transfer_stop'].fillna(False)

In [50]:
merged_df

,Unnamed: 0,route_id,agency_id,route_short_name,route_type,route_type_name,agency_name,service_id,trip_id,trip_headsign,...,station_point,buffer_zone,scheduled_arrival_time,scheduled_departure_time,arrival_delay,departure_delay,service,date,weekday,transfer_stop
0,0,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,POINT (7.158347000000002 50.680336000000004),"POLYGON ((7.161095563015469 50.68066886560957,...",2023-07-01 05:17:00,2023-07-01 05:17:00,NaN,0 days,1,2023-07-01,Saturday,True
1,1,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,POINT (7.159553000000001 50.68392),"POLYGON ((7.162301786678823 50.68425285400936,...",2023-07-01 05:18:00,2023-07-01 05:18:00,0 days,0 days,1,2023-07-01,Saturday,True
2,2,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,POINT (7.153876000000002 50.687850999999995),POLYGON ((7.1566249492160345 50.68818400637061...,2023-07-01 05:19:00,2023-07-01 05:19:00,0 days,0 days,1,2023-07-01,Saturday,False
3,3,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,POINT (7.1482320000000055 50.691395),"POLYGON ((7.150981089535626 50.69172815614405,...",2023-07-01 05:20:00,2023-07-01 05:20:00,0 days,0 days,1,2023-07-01,Saturday,False
4,4,16,1,16,0,Tram,KVB Kölner Verkehrs-Betriebe AG,130,644-16-001-371.1.11:051700-19-2_B87C6E2C-D2B2-...,Buschdorf,...,POINT (7.141498000000006 50.69677399999999),POLYGON ((7.1442473242318325 50.69710734026554...,2023-07-01 05:21:00,2023-07-01 05:21:00,0 days,0 days,1,2023-07-01,Saturday,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71174467,71174467,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,POINT (7.048342000000008 50.718540999999995),"POLYGON ((7.051091489774908 50.71887664006563,...",2023-07-31 20:36:00,2023-07-31 20:36:00,NaN,NaN,1,2023-07-31,Monday,False
71174468,71174468,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,POINT (7.044228999999996 50.722075000000004),POLYGON ((7.0469786475110325 50.72241075379365...,2023-07-31 20:37:00,2023-07-31 20:37:00,NaN,NaN,1,2023-07-31,Monday,False
71174469,71174469,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,POINT (7.045797000000002 50.723193999999985),"POLYGON ((7.048546731723949 50.72352972210798,...",2023-07-31 20:38:00,2023-07-31 20:38:00,NaN,NaN,1,2023-07-31,Monday,False
71174470,71174470,884,12,884,3,Bus,RVK Regionalverkehr Köln GmbH NL Meckenheim,27690,60-884-012-3332.2.22:202100-16-1_E4B90C8F-59DE...,Lessenich Sportplatz,...,POINT (7.047697000000004 50.722637),POLYGON ((7.050446721797707 50.722972674659836...,2023-07-31 20:38:00,2023-07-31 20:38:00,NaN,NaN,1,2023-07-31,Monday,False


In [51]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71174472 entries, 0 to 71174471
Data columns (total 29 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   Unnamed: 0                int64         
 1   route_id                  int64         
 2   agency_id                 int64         
 3   route_short_name          object        
 4   route_type                int64         
 5   route_type_name           object        
 6   agency_name               object        
 7   service_id                int64         
 8   trip_id                   object        
 9   trip_headsign             object        
 10  direction_id              int64         
 11  shape_id                  int64         
 12  stop_id                   int64         
 13  actual_arrival_time       datetime64[ns]
 14  actual_departure_time     datetime64[ns]
 15  vrs_timestamp             int64         
 16  stop_sequence             int64         
 17  stop_h

In [52]:
from shapely.wkt import loads

In [53]:
merged_df['station_point'] = merged_df['station_point'].apply(shapely.wkt.loads)

In [54]:
gdf = gpd.GeoDataFrame(merged_df, geometry='station_point')

In [55]:
gdf['route_short_name'] = gdf['route_short_name'].astype(str)

In [ ]:
#gdf.to_parquet('../Data/10_second_merge_NEU_NEU.parquet')

In [57]:
#gdf.to_csv('../Data/10_second_merge_NEU_NEU.csv')
gdf.to_csv('../Data/09_second_merge.csv')